In [ ]:
import json
import time

import requests
from PIL import Image  # noqa: F401
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC  # noqa: N812
from selenium.webdriver.support.ui import WebDriverWait


In [ ]:
#login_url = "http://pdharvard.egreenapple.com.ezp-prod1.hul.harvard.edu/index2.html"
login_url = 'https://www.pin1.harvard.edu/cas/login?service=https%3A%2F%2Fhollis.harvard.edu%3A443%2Fprimo_library%2Flibweb%2FcasRedirect%3FauthenticationProfile%3DHarvard+Primo%26targetURL%3Dhttps%253A%252F%252Fhollis.harvard.edu%252Fprimo-explore%252Fsearch%253Fvid%253DHVD2%2526sortby%253Drank%2526lang%253Den_US%2526from-new-ui%253D1%2526authenticationProfile%253DHarvard%252BPrimo%26institution%3D01HVD'
base_url = 'https://images.hollis.harvard.edu/primo-explore/search?query=any,contains,trade%20cards&tab=default_tab&search_scope=default_scope&vid=HVD_IMAGES&lang=en_US&offset=0#searchResultList'

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)


### Run the next two cells only if working off-campus

In [ ]:
# off-campus setup
driver.get(login_url)

### Log in before running next cell

In [ ]:
# Start a session in requests
s = requests.session()

# Set cookies in requests session from selenium session
for cookie in driver.get_cookies():
    c = {cookie['name']: cookie['value']}
    s.cookies.update(c)

# Show the new session cookies
# s.cookies.get_dict()

# Set session headers to look like Selenium session and add the referer url
user_agent = driver.execute_script('return navigator.userAgent;')
headers = {'User-Agent': user_agent, 'Referer': login_url}
s.headers.update(headers)

# Show the new headers
# s.headers


In [ ]:
# driver.switch_to.window(original_window)
driver.get(base_url)

In [ ]:
# set starting point | e.g. if process is interrupted for some reason
starting_page = 1
starting_image = 0 # 0-based count

links = []
problems = []
metadata = []
page_no = starting_page
original_window = driver.current_window_handle
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 30)

# need to go to starting page if it's not the first
if starting_page > 1:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f'button[aria-label="Go to page {starting_page}"]')))
    goto_page = driver.find_elements(By.CSS_SELECTOR, f'button[aria-label="Go to page {starting_page}"]')
    goto_page[0].click()
    time.sleep(10)

while page_no <= 29:  # noqa: PLR2004
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.thumbnail')))
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'a.thumbnail')
    no_thumbs = len(image_elements)

    for idx in range(starting_image, no_thumbs):
        try:
            popup = False

            if EC.staleness_of(image_elements[idx]):
                image_elements = driver.find_elements(By.CSS_SELECTOR, 'a.thumbnail')

            try:
                wait.until(EC.element_to_be_clickable(image_elements[idx]))
                image_elements[idx].click()

            except ElementClickInterceptedException:
                driver.switch_to.default_content()
                dialogue = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'prm-brief-result-container')))
                dialogue.click()
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                image_elements[idx].click()

            except:  # noqa: E722
                driver.get(base_url)
                time.sleep(10)
                image_elements = driver.find_elements(By.CSS_SELECTOR, 'a.thumbnail')
                image_elements[idx].click()

            try:
                test = image_elements[idx].find_elements(By.CSS_SELECTOR, '.nophoto')
                has_photo = not len(test)
            except:  # noqa: E722
                has_photo = True

            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'md-dialog-content')))

            detail_rows = driver.find_elements(By.CSS_SELECTOR, '#details div.spaced-rows div[layout="row"')
            data = {}
            for row in detail_rows:
                title = row.find_elements(By.CSS_SELECTOR, 'span[data-details-label]')[0].text
                value_list = row.find_elements(By.CSS_SELECTOR, 'div[role="listitem"]')
                values = [v.text for v in value_list]
                if len(values):
                    data[title] = values if len(values) > 1 else values[0]

            metadata.append(data)

            if has_photo:
                link_list = []
                target = wait.until(EC.any_of(
                    EC.presence_of_element_located((By.ID, 'iframe')),
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'md-grid-list a.thumbnail')),
                ))

                if target.tag_name == 'iframe':
                    driver.switch_to.frame('iframe')
                    img_count = 1
                else:
                    popup = True
                    images = driver.find_elements(By.CSS_SELECTOR, 'md-grid-list a.thumbnail')

                    try:
                        wait.until(EC.element_to_be_clickable(images[0]))
                        images[0].click()
                    except:  # noqa: E722
                        images[1].click()

                    wait.until(EC.number_of_windows_to_be(2))

                    for window_handle in driver.window_handles:
                        if window_handle != original_window:
                            driver.switch_to.window(window_handle)
                            break

                    img_count = int(driver.find_element(By.CSS_SELECTOR, '.imageNav label.label').text[4:6])
                    next_button = driver.find_element(By.CSS_SELECTOR, '.imageNav button[aria-label*="Next"]')
                    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'iframe')))


                count = 0
                while count < img_count:
                    count = count + 1
                    try:
                        try:
                            button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.mirador-icon-download a')))
                            button.click()
                            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li[title*="Download IIIF"] a')))
                        except:  # noqa: E722
                            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li[title*="Download IIIF"] a')))

                    except Exception as e:  # noqa: BLE001
                        problems.append(f'Error A: Page {page_no}, image {idx + 1}: {e.msg}\n{e.stacktrace}')
                        print(f'Error: Page {page_no}, image {idx + 1}: {e.msg[0:50]}')

                    else:
                        manifest = driver.find_elements(By.CSS_SELECTOR, 'li[title*="Download IIIF"] a')
                        link_list.append(manifest[0].get_attribute('href'))

                    if img_count > 1:
                        driver.switch_to.default_content()
                        next_button.click()
                        time.sleep(5)
                        wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'iframe')))

                if popup:
                    driver.close();
                    driver.switch_to.window(original_window)

                driver.switch_to.default_content()
                dialogue = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'prm-brief-result-container')))
                dialogue.click()
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

                if len(link_list):
                    links.append(link_list)

            else:
                driver.switch_to.default_content()
                dialogue = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'prm-brief-result-container')))
                dialogue.click()
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

        except Exception as e:  # noqa: BLE001
            if len(driver.window_handles) > 1:
                for window_handle in driver.window_handles:
                    if window_handle != original_window:
                        driver.switch_to.window(window_handle)
                        driver.close();

                driver.switch_to.window(original_window)

            driver.switch_to.default_content()
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            problems.append(f'Error B: Page {page_no}, image {idx + 1}: {e.msg}\n{e.stacktrace}')
            print(f'Error: Page {page_no}, image {idx + 1}: {e.msg[0:50]}')

    print(f'Page {page_no} completed.')

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button[aria-label="Next page"]')))
    next_page = driver.find_elements(By.CSS_SELECTOR, 'button[aria-label="Next page"]')
    next_page[0].click()
    time.sleep(5)
    starting_image = 0 # reset starting image for pages going forwards
    page_no = page_no + 1


In [ ]:
with open('links.json', 'w') as file:
    file.write(json.dumps(links))

with open('metadata.json', 'w') as file:
    file.write(json.dumps(metadata))

with open('problems.txt', 'w') as file:
    file.write(json.dumps(problems))


In [ ]:
for p in problems:
    print(p)

In [ ]:
for img, lset in enumerate(links):
    for idx, link in enumerate(lset):
        img_id = link[link.rfind(':') + 1:]
        response = s.get(f'https://ids.lib.harvard.edu/ids/iiif/{img_id}/full/full/0/default.jpg')
        with open(f'images/{img}-{idx}-img.jpg', 'wb') as outfile:
            outfile.write(response.content)
